In [1]:
import pandas as pd
import numpy as np

In [2]:
file_path = 'clensed_data.pkl'
df = pd.read_pickle(file_path)

print(" Data loaded successfully.")


 Data loaded successfully.


In [3]:
print(df.info(verbose=True, max_cols=30))

<class 'pandas.core.frame.DataFrame'>
Index: 2013945 entries, 0 to 17284449
Data columns (total 23 columns):
 #   Column              Dtype         
---  ------              -----         
 0   Unnamed: 0          int64         
 1   ssn                 string        
 2   cc_num              int64         
 3   city                string        
 4   zip                 int64         
 5   lat                 float64       
 6   long                float64       
 7   city_pop            float64       
 8   job                 string        
 9   dob                 datetime64[ns]
 10  acct_num            int64         
 11  profile             string        
 12  trans_num           string        
 13  trans_date          datetime64[ns]
 14  category            string        
 15  amt                 float64       
 16  is_fraud            int64         
 17  merchant            string        
 18  merch_lat           float64       
 19  merch_long          float64       
 20  is_mal

##Date Features

In [4]:
current_year = 2020

df['TX_MONTH'] = df['trans_date'].dt.month
print("TX_MONTH feature created.")
df['TX_DAY'] = df['trans_date'].dt.day
print("TX_DAY feature created.")
df['TX_HOUR'] = df['trans_timestamp'].dt.hour
print("TX_HOUR feature created.")

df['DOB_YEAR']= df['dob'].dt.year
print("DOB_YEAR feature created.")
df['DOB_MONTH']= df['dob'].dt.month
print("DOB_MONTH feature created.")


df['age'] = current_year - df['DOB_YEAR']
print("Age feature created.")
df['IS_WEEKEND'] = np.where(df['trans_date'].dt.dayofweek >= 5, 1, 0)
print("IS_WEEKEND feature created.")

TX_MONTH feature created.
TX_DAY feature created.
TX_HOUR feature created.
DOB_YEAR feature created.
DOB_MONTH feature created.
Age feature created.
IS_WEEKEND feature created.


##Time since last transaction

In [5]:
df.sort_values(by=['cc_num', 'trans_timestamp'], inplace=True)

df['TIME_SINCE_LAST_TX'] = (
    df.groupby('cc_num')['trans_timestamp']
      .diff()
      .dt.total_seconds()
      .fillna(0)
)

print("'TIME_SINCE_LAST_TX' (in seconds) created.")

'TIME_SINCE_LAST_TX' (in seconds) created.


##Velocity Features

In [6]:
df = df.reset_index(drop=True)
df.sort_values(by=['cc_num', 'trans_timestamp'], inplace=True)

WINDOWS = ['1h', '24h', '7d'] 

for window in WINDOWS:
    tx_count_col = f'TX_COUNT_{window}'
    amt_max_col = f'AMT_MAX_{window}'
    amt_avg_col = f'AMT_AVG_{window}'

    df[tx_count_col] = 0.0
    df[amt_max_col] = 0.0
    df[amt_avg_col] = 0.0

    for cc_val, grp in df.groupby('cc_num', sort=False):
        s = grp.set_index('trans_timestamp')['amt']

        cnt = s.rolling(window=window).count().shift(1).fillna(0).values
        mx = s.rolling(window=window).max().shift(1).fillna(0).values
        avg = s.rolling(window=window).mean().shift(1).fillna(0).values

        df.loc[grp.index, tx_count_col] = cnt
        df.loc[grp.index, amt_max_col] = mx
        df.loc[grp.index, amt_avg_col] = avg

print("Rolling Velocity Features (TX_COUNT_*, AMT_MAX_*, AMT_AVG_*) created.")

Rolling Velocity Features (TX_COUNT_*, AMT_MAX_*, AMT_AVG_*) created.


##Relative amount

In [7]:
# overall lifetime average amount
user_avg_amt = df.groupby('cc_num')['amt'].mean().rename('USER_LIFETIME_AVG_AMT')

df = df.merge(user_avg_amt, on='cc_num', how='left')

df['AMT_vs_USER_AVG'] = df['amt'] / df['USER_LIFETIME_AVG_AMT']
df['AMT_vs_USER_AVG'] = df['AMT_vs_USER_AVG'].fillna(0)

df.drop(columns=['USER_LIFETIME_AVG_AMT'], inplace=True)

print("Relative Amount Feature (AMT_vs_USER_AVG) created.")

Relative Amount Feature (AMT_vs_USER_AVG) created.


##Geospatial Features:

In [8]:
from geopy.distance import great_circle

def calculate_great_circle_distance(row):
    """Calculates distance in meters between customer home and merchant."""
    try:
        customer_loc = (row['lat'], row['long'])
        merchant_loc = (row['merch_lat'], row['merch_long'])
        return great_circle(customer_loc, merchant_loc).meters
    except Exception:
        return 0 # Return 0 for bad coordinates

df['DIST_HOME_MERCH'] = df.apply(calculate_great_circle_distance, axis=1)

df['DIST_HOME_MERCH_LOG'] = np.log1p(df['DIST_HOME_MERCH'])
df.drop(columns=['DIST_HOME_MERCH'], inplace=True)

print("Geospatial Distance Feature (DIST_HOME_MERCH_LOG) created.")

Geospatial Distance Feature (DIST_HOME_MERCH_LOG) created.


In [9]:
profile_components = df['profile'].str.split('_', expand=True)
df['PROFILE_GEO_TYPE'] = profile_components[3]
print("'PROFILE_GEO_TYPE' created.")

'PROFILE_GEO_TYPE' created.


##Transation Velocity Features

In [10]:
df = df.reset_index(drop=True)

df.sort_values(by=['cc_num', 'trans_timestamp'], inplace=True)

WINDOWS = ['1h', '24h', '7d']

for window in WINDOWS:
    print(f"Calculating features for window: {window}")

    tx_count_col = f'TX_COUNT_{window}'
    amt_max_col = f'AMT_MAX_{window}'
    amt_avg_col = f'AMT_AVG_{window}'

    df[tx_count_col] = 0.0
    df[amt_max_col] = 0.0
    df[amt_avg_col] = 0.0

    for cc_val, grp in df.groupby('cc_num', sort=False):
        s = grp.set_index('trans_timestamp')['amt']

        cnt = s.rolling(window=window).count().shift(1).fillna(0).values
        mx = s.rolling(window=window).max().shift(1).fillna(0).values
        avg = s.rolling(window=window).mean().shift(1).fillna(0).values

        df.loc[grp.index, tx_count_col] = cnt
        df.loc[grp.index, amt_max_col] = mx
        df.loc[grp.index, amt_avg_col] = avg

print("Rolling Velocity Features created successfully.")

Calculating features for window: 1h
Calculating features for window: 24h
Calculating features for window: 7d
Rolling Velocity Features created successfully.


In [11]:
#Historical average of the is_fraud flag, grouped by zip code.

df.sort_values(by=['trans_timestamp'], inplace=True)
df = df.reset_index(drop=True)

def zip_card_count_1d(group):
    s = group.set_index('trans_timestamp')['cc_num']
    res = s.rolling('24h').apply(lambda x: x.nunique(), raw=False).shift(1)
    return pd.Series(res.values, index=group.index).fillna(0)

df['ZIP_CARD_COUNT_1D'] = df.groupby('zip', group_keys=False).apply(zip_card_count_1d, include_groups=False).astype(float)

print("'ZIP_CARD_COUNT_1D' (Unique Cards in 24h) created.")

'ZIP_CARD_COUNT_1D' (Unique Cards in 24h) created.


##SSN-Based Features

In [12]:
ssn_acct_counts = df.groupby('ssn')['acct_num'].nunique().rename('UNIQUE_ACCT_COUNT')
df = df.merge(ssn_acct_counts, on='ssn', how='left')
# 1 if the SSN is tied to more than one account, 0 otherwise
df['SSN_SHARED_FLAG'] = np.where(df['UNIQUE_ACCT_COUNT'] > 1, 1, 0)

df.drop(columns=['UNIQUE_ACCT_COUNT'], inplace=True)

print("'SSN_SHARED_FLAG' created.")

'SSN_SHARED_FLAG' created.


In [13]:
df = df.reset_index(drop=True)
df['SSN_COUNT_1D'] = 0.0

for ssn_val, grp in df.groupby('ssn', sort=False):
    sorted_grp = grp.sort_values('trans_timestamp')
    s = sorted_grp.set_index('trans_timestamp')['amt']
    cnt = s.rolling(window='24h').count().shift(1).fillna(0).values
    df.loc[sorted_grp.index, 'SSN_COUNT_1D'] = cnt

print("'SSN_COUNT_1D' (count in last 24 hours) created.")

'SSN_COUNT_1D' (count in last 24 hours) created.


##CC_NUM Features

In [14]:
#finds if the cc number had previous frauds

df.sort_values(by=['cc_num', 'trans_timestamp'], inplace=True)

df['CC_CUM_FRAUD'] = df.groupby('cc_num')['is_fraud'].cumsum()
df['CC_PREV_FRAUD'] = df['CC_CUM_FRAUD'].shift(1).fillna(0)

df.drop(columns=['CC_CUM_FRAUD'], inplace=True)

print("'CC_PREV_FRAUD' (Historical Fraud Count) created.")

'CC_PREV_FRAUD' (Historical Fraud Count) created.


In [15]:
#card type
df['CC_BIN'] = df['cc_num'][:6]

print("'CC_BIN' (Categorical BIN) created.")

'CC_BIN' (Categorical BIN) created.


In [16]:
# total count of transactions for each cc_num
cc_lifetime_counts = df.groupby('cc_num')['amt'].count().rename('CC_COUNT_LIFETIME')
df = df.merge(cc_lifetime_counts, on='cc_num', how='left')

print("'CC_COUNT_LIFETIME' created.")

'CC_COUNT_LIFETIME' created.


##Binning
#using quantile binning to deal to highly skewed data

In [17]:
def create_and_drop_bins(df: pd.DataFrame, features_to_bin: list, num_bins: int = 5) -> pd.DataFrame:
    """
    Applies quantile binning to a list of numerical features, creates new 
    categorical columns, and drops the original numerical columns.

    Args:
        df (pd.DataFrame): The input DataFrame.
        features_to_bin (list): List of numerical column names to bin.
        num_bins (int): The number of quantile bins to create.

    Returns:
        pd.DataFrame: The DataFrame with new binned features and dropped originals.
    """
    binned_features = []
    dropped_originals = []
    
    print(f"Starting Quantile Binning for {features_to_bin} into {num_bins} bins.")

    for col in features_to_bin:
        new_col_name = f'{col}_BIN'
        
        if col not in df.columns:
            print(f"⚠️ Warning: Column '{col}' not found. Skipping.")
            continue
            
        try:
            #Creates bins with equal number of records
            df[new_col_name] = pd.qcut(
                df[col], 
                q=num_bins, 
                labels=False, 
                duplicates='drop' 
            ).astype('category').astype(str) 
            
            binned_features.append(new_col_name)
            dropped_originals.append(col)
            print(f"  - ✅ '{col}' successfully binned into {df[new_col_name].nunique()} segments.")
            
        except ValueError as e:
            print(f"  - ❌ Could not bin '{col}' into {num_bins} bins. Error: {e}")
            continue

    df.drop(columns=dropped_originals, inplace=True, errors='ignore') 
    
    print("\n--- Binning Summary ---")
    print(f" Binned Features Created: {binned_features}")
    print(f" Original Numerical Features Dropped: {dropped_originals}")
    
    return df



In [18]:
BINNING_CANDIDATES = [
    'amt', 
    'city_pop', 
    'TIME_SINCE_LAST_TX',
    'age',
    'AMT_vs_USER_AVG', 
    'ZIP_CARD_COUNT_1D',
    'AMT_AVG_7d', 
    'CC_COUNT_LIFETIME'
]

df = create_and_drop_bins(df, BINNING_CANDIDATES, num_bins=5)

Starting Quantile Binning for ['amt', 'city_pop', 'TIME_SINCE_LAST_TX', 'age', 'AMT_vs_USER_AVG', 'ZIP_CARD_COUNT_1D', 'AMT_AVG_7d', 'CC_COUNT_LIFETIME'] into 5 bins.
  - ✅ 'amt' successfully binned into 5 segments.
  - ✅ 'city_pop' successfully binned into 5 segments.
  - ✅ 'TIME_SINCE_LAST_TX' successfully binned into 5 segments.
  - ✅ 'age' successfully binned into 5 segments.
  - ✅ 'AMT_vs_USER_AVG' successfully binned into 5 segments.
  - ✅ 'ZIP_CARD_COUNT_1D' successfully binned into 5 segments.
  - ✅ 'AMT_AVG_7d' successfully binned into 5 segments.
  - ✅ 'CC_COUNT_LIFETIME' successfully binned into 5 segments.

--- Binning Summary ---
 Binned Features Created: ['amt_BIN', 'city_pop_BIN', 'TIME_SINCE_LAST_TX_BIN', 'age_BIN', 'AMT_vs_USER_AVG_BIN', 'ZIP_CARD_COUNT_1D_BIN', 'AMT_AVG_7d_BIN', 'CC_COUNT_LIFETIME_BIN']
 Original Numerical Features Dropped: ['amt', 'city_pop', 'TIME_SINCE_LAST_TX', 'age', 'AMT_vs_USER_AVG', 'ZIP_CARD_COUNT_1D', 'AMT_AVG_7d', 'CC_COUNT_LIFETIME']


##Dropping original and useless features

In [19]:
df.drop(columns=['profile','zip', 'long', 'lat', 'Unnamed: 0', 'cc_num', 'dob', 'acct_num', 'trans_num'
                 ,'trans_date','merch_lat', 'merch_long', 'trans_timestamp','ssn','DOB_YEAR', 'DOB_MONTH'], inplace=True)

In [20]:
df.describe()

,is_fraud,is_male,IS_ANY_OUTLIER_IQR,TX_MONTH,TX_DAY,TX_HOUR,IS_WEEKEND,TX_COUNT_1h,AMT_MAX_1h,AMT_AVG_1h,TX_COUNT_24h,AMT_MAX_24h,AMT_AVG_24h,TX_COUNT_7d,AMT_MAX_7d,DIST_HOME_MERCH_LOG,SSN_SHARED_FLAG,SSN_COUNT_1D,CC_PREV_FRAUD,CC_BIN
count,2.013945e+06,2.013945e+06,2.013945e+06,2.013945e+06,2.013945e+06,2.013945e+06,2.013945e+06,2.013945e+06,2.013945e+06,2.013945e+06,2.013945e+06,2.013945e+06,2.013945e+06,2.013945e+06,2.013945e+06,2.013945e+06,2013945.0,2.013945e+06,2.013945e+06,6.000000e+00
mean,5.409284e-03,4.900392e-01,2.168748e-01,7.119759e+00,1.585131e+01,1.672458e+01,3.993048e-01,1.333607e+00,5.873726e+01,5.061403e+01,5.032191e+00,1.077685e+02,5.057659e+01,2.686795e+01,1.634760e+02,1.115439e+01,0.0,5.032191e+00,2.761014e+00,6.040566e+10
std,7.334866e-02,4.999009e-01,4.121168e-01,3.423812e+00,8.837511e+00,4.570042e+00,4.897557e-01,6.252567e-01,5.434005e+01,4.786279e+01,3.253542e+00,5.661685e+01,2.990266e+01,1.488059e+01,3.022265e+01,5.166862e-01,0.0,3.253542e+00,4.686068e+00,0.000000e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.842894e+00,0.0,0.000000e+00,0.000000e+00,6.040566e+10
25%,0.000000e+00,0.000000e+00,0.000000e+00,4.000000e+00,8.000000e+00,1.400000e+01,0.000000e+00,1.000000e+00,9.980000e+00,9.680000e+00,3.000000e+00,6.251000e+01,2.945000e+01,1.600000e+01,1.596500e+02,1.094452e+01,0.0,3.000000e+00,0.000000e+00,6.040566e+10
50%,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e+00,1.600000e+01,1.700000e+01,0.000000e+00,1.000000e+00,4.320000e+01,3.673000e+01,4.000000e+00,1.057400e+02,4.722000e+01,2.400000e+01,1.793950e+02,1.129042e+01,0.0,4.000000e+00,0.000000e+00,6.040566e+10
75%,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+01,2.400000e+01,2.000000e+01,1.000000e+00,2.000000e+00,8.926000e+01,7.570000e+01,7.000000e+00,1.726700e+02,6.683500e+01,3.500000e+01,1.793950e+02,1.151035e+01,0.0,7.000000e+00,7.000000e+00,6.040566e+10
max,1.000000e+00,1.000000e+00,1.000000e+00,1.200000e+01,3.100000e+01,2.300000e+01,1.000000e+00,9.000000e+00,1.793950e+02,1.793950e+02,3.900000e+01,1.793950e+02,1.793950e+02,1.090000e+02,1.793950e+02,1.188653e+01,0.0,3.900000e+01,1.600000e+01,6.040566e+10


##Hot One Encoding

In [21]:
CATEGORICAL_FEATURES = ['category','city', 'TX_MONTH', 'TX_HOUR','TX_DAY', 'amt_BIN', 'city_pop_BIN', 'TIME_SINCE_LAST_TX_BIN', 'age_BIN',
                         'merchant', 'job', 'PROFILE_GEO_TYPE', 'CC_BIN' ]

df = pd.get_dummies(
    df,
    columns=CATEGORICAL_FEATURES,
    prefix=CATEGORICAL_FEATURES,
    drop_first=True 
)
print(f"Encoding Complete. Final feature count: {df.shape[1] - 1}")

Encoding Complete. Final feature count: 173


##Standardization

In [22]:

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


X = df.drop(columns=['is_fraud'])
y = df['is_fraud']

ALL_FEATURES = X.columns.tolist()

print("Starting final Standardization for all features...")
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, index=X.index, columns=ALL_FEATURES)
df = pd.concat([X_scaled, y], axis=1)
print("Final Standardization Complete.")

Starting final Standardization for all features...
Final Standardization Complete.


##Unbalanced Data fitting

In [23]:

X = df.drop(columns=['is_fraud'])
y = df['is_fraud'] 

total_samples = len(df)


X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, 
    test_size=0.3, 
    random_state=42, 
    stratify=y 
)


X_dev, X_test, y_dev, y_test = train_test_split(
    X_temp, y_temp, 
    test_size=0.5,
    random_state=42, 
    stratify=y_temp 
)

train_percent = (X_train.shape[0] / total_samples) * 100
dev_percent = (X_dev.shape[0] / total_samples) * 100
test_percent = (X_test.shape[0] / total_samples) * 100

print(f"Total Samples: {total_samples:,}")
print(f"Training set size: {X_train.shape[0]:,} ({train_percent:.2f}%)")
print(f"Validation (Dev) set size: {X_dev.shape[0]:,} ({dev_percent:.2f}%)")
print(f"Testing set size: {X_test.shape[0]:,} ({test_percent:.2f}%)")
print("\nData split successfully with stratification.")

Total Samples: 2,013,945
Training set size: 1,409,761 (70.00%)
Validation (Dev) set size: 302,092 (15.00%)
Testing set size: 302,092 (15.00%)

Data split successfully with stratification.


##Feature Selection:


In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier


imbalance_ratio = (y_train == 0).sum() / (y_train == 1).sum()
class_weights = {0: 1, 1: imbalance_ratio}

models = {
    # 1. Lasso (L1 penalty)
    'Lasso_L1': LogisticRegression(penalty='l1', solver='saga', C=0.01, class_weight=class_weights, random_state=42, max_iter=5000),
    
    # 2. Ridge (L2 penalty)
    'Ridge_L2': LogisticRegression(penalty='l2', solver='saga', C=0.01, class_weight=class_weights, random_state=42, max_iter=5000),
    
    # 3. Linear SVM (L1 penalty) - rmoved due to long training times
    #'SVM_L1': LinearSVC(C=0.01, penalty="l1", dual=False, class_weight=class_weights, random_state=42, max_iter=2000),
    
    # 4. Gradient Boosting
    'GradientBoost': GradientBoostingClassifier(n_estimators=50, max_depth=4, random_state=42),
    
    # 5. Random Forest
    'RandomForest': RandomForestClassifier(n_estimators=50, max_depth=8, class_weight=class_weights, random_state=42, n_jobs=-1),
}

selection_results = {}
print("Starting Consensus Feature Selection...")

for name, model in models.items():
    print(f"-> Fitting {name}...")
    
    model.fit(X_train, y_train)

    if hasattr(model, 'coef_'):
        coef = model.coef_[0] if model.coef_.ndim > 1 else model.coef_
        selected = (np.abs(coef) > 1e-4).astype(int)
    elif hasattr(model, 'feature_importances_'):
        selected = (model.feature_importances_ > 1e-4).astype(int)
    else:
        selected = np.zeros(X_train.shape[1]).astype(int)
    
    selection_results[name] = selected

selection_df = pd.DataFrame(selection_results, index=X_train.columns)
selection_df.index.name = 'Feature'

selection_df['Sum'] = selection_df.sum(axis=1)
selection_df = selection_df.sort_values(by='Sum', ascending=False)

TOP_N_FEATURES = 20
print(f"Filtering down to the absolute top {TOP_N_FEATURES} features...")

final_selected_features_df = selection_df.sort_values(
    by='Sum', 
    ascending=False
).head(TOP_N_FEATURES)

final_selected_features = final_selected_features_df.index.tolist()

print(f"✅ Final feature count selected: {len(final_selected_features)}")
print("\nTop 20 Features Selected:")
print(final_selected_features_df[['Sum']].to_markdown())


Starting Consensus Feature Selection...
-> Fitting Lasso_L1...


/opt/anaconda/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


-> Fitting Ridge_L2...


/opt/anaconda/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


-> Fitting GradientBoost...
-> Fitting RandomForest...
Filtering down to the absolute top 20 features...
✅ Final feature count selected: 20

Top 20 Features Selected:
| Feature                |   Sum |
|:-----------------------|------:|
| IS_ANY_OUTLIER_IQR     |     4 |
| AMT_MAX_1h             |     4 |
| TX_COUNT_1h            |     4 |
| TX_COUNT_24h           |     4 |
| AMT_AVG_1h             |     4 |
| TX_COUNT_7d            |     4 |
| DIST_HOME_MERCH_LOG    |     4 |
| AMT_MAX_24h            |     4 |
| AMT_AVG_24h            |     4 |
| AMT_AVG_7d_BIN         |     4 |
| category_food_dining   |     4 |
| CC_PREV_FRAUD          |     4 |
| SSN_COUNT_1D           |     4 |
| category_misc_net      |     4 |
| category_travel        |     4 |
| category_shopping_pos  |     4 |
| category_shopping_net  |     4 |
| category_gas_transport |     4 |
| category_misc_pos      |     4 |
| category_grocery_pos   |     4 |


In [25]:
SELECTED_FEATURES = final_selected_features

X_train_filtered = X_train[SELECTED_FEATURES]
X_dev_filtered = X_dev[SELECTED_FEATURES]
X_test_filtered = X_test[SELECTED_FEATURES]

np.save('selected_features.npy', np.array(SELECTED_FEATURES))

df_train_final = pd.concat([X_train_filtered, y_train], axis=1)
df_dev_final = pd.concat([X_dev_filtered, y_dev], axis=1)
df_test_final = pd.concat([X_test_filtered, y_test], axis=1)

df_train_final.to_pickle('train_set_final_filtered.pkl')
df_dev_final.to_pickle('dev_set_final_filtered.pkl')
df_test_final.to_pickle('test_set_final_filtered.pkl')

print("\n Final, optimized, and filtered data sets saved for Modeling.")



 Final, optimized, and filtered data sets saved for Modeling.
